In [1]:
#include <cmath>
#include <list>
#include <xtensor/xarray.hpp>
#include <iostream>

double r(const int& i, 
        const std::list<int>& G, 
        const xt::xarray<double>& dist, 
        const int& n) 
{    
    double max = 0;
    
    for(int j : G){
        double dij;
        if (i==j)
            dij = 0;
        else if(i < j)
            dij = dist[n*i+j-((i+2)*(i+1))/2];
        else
            dij = dist[n*j+i-((j+2)*(j+1))/2];
        
        max =  dij > max ? dij : max;
    }
    return max;
}


In [2]:
#include <xtensor/xarray.hpp>
#include <xtensor/xnpy.hpp>
#include <list>
#include <unordered_map>
#include <algorithm>
#include <cmath>
#include <limits>
#include <iterator>

using ClusterMap = std::unordered_map<int, std::list<int>>;

// auto dist = xt::load_npy<double>("tangent_distances.npy");

auto dist = xt::load_npy<double>("toy.npy");
int n = std::round((std::sqrt(8*dist.size() + 1) + 1)/2);

xt::xarray<double> Z = xt::xarray<double>::from_shape({static_cast<unsigned long>(n-1), 4});
ClusterMap clusters;

for (int i=0 ; i<n ; i++)
    clusters[i] = std::list<int>({i});

for (size_t i=0 ; i<n-1 ; i++)
{
    double min_d = std::numeric_limits<double>::infinity();
    xt::xarray<double> to_merge;
    
    for(ClusterMap::iterator it=clusters.begin(); 
        it!=clusters.end() ; it++){
        
        for(ClusterMap::iterator jt=clusters.begin(); 
            jt!=it ; jt++){
            
            const int& idxG = it->first;
            const int& idxH = jt->first;
            
            std::list<int>& GH = it->second;
            std::list<int>& H = jt->second;
            
            GH.merge(H);
            
            double d = std::numeric_limits<double>::infinity();
            for (const int& x : GH){
                double minimax = r(x, GH, dist, n);
                d = minimax < d ? minimax : d;
            }
            if (d < min_d){
                min_d = d;
                to_merge = {(double)idxG, (double)idxH, d, (double)GH.size()};
            }       
        }
    }     
    
    for (int k=0 ; k<4 ; k++)
        Z(i, k) = to_merge[k];
    
    double idxG = to_merge[0];
    double idxH = to_merge[1];
    
    std::list<int> new_cluster = clusters[idxG];
    new_cluster.merge(clusters[(int)idxH]);
    
    clusters[n+i] = new_cluster;
    clusters.erase(idxG);
    clusters.erase(idxH);
}

xt::dump_npy("Z_tan_minimax.npy", Z);

In [2]:
#include <xtensor/xarray.hpp>
#include <xtensor/xnpy.hpp>
#include <unordered_set>
#include <unordered_map>
#include <algorithm>
#include <cmath>
#include <limits>
#include <iterator>

using ClusterMap = std::unordered_map<int, std::unordered_set<int>>;

// auto dist = xt::load_npy<double>("tangent_distances.npy");

auto dist = xt::load_npy<double>("toy.npy");
int n = std::round((std::sqrt(8*dist.size() + 1) + 1)/2);

xt::xarray<double> Z = xt::xarray<double>::from_shape({static_cast<unsigned long>(n-1), 4});
ClusterMap clusters;

for (int i=0 ; i<n ; i++)
    clusters[i] = std::unordered_set<int>({i});

for (size_t i=0 ; i<n-1 ; i++)
{
    double min_d = std::numeric_limits<double>::infinity();
    xt::xarray<double> to_merge;
    
    for(ClusterMap::iterator it=clusters.begin(); 
        it!=clusters.end() ; it++){
        
        for(ClusterMap::iterator jt=clusters.begin(); 
            jt!=it ; jt++){
            
            const int& idxG = it->first;
            const int& idxH = jt->first;
            
            std::unordered_set<int>& GH = it->second;
            std::unordered_set<int>& H = jt->second;
            
            GH.merge(H);
            
            double d = std::numeric_limits<double>::infinity();
            for (const int& x : GH){
                double minimax = r(x, GH, dist, n);
                d = minimax < d ? minimax : d;
            }
            if (d < min_d){
                min_d = d;
                to_merge = {(double)idxG, (double)idxH, d, (double)GH.size()};
            }       
        }
    }     
    
    for (int k=0 ; k<4 ; k++)
        Z(i, k) = to_merge[k];
    
    double idxG = to_merge[0];
    double idxH = to_merge[1];
    
    std::unordered_set<int> new_cluster = clusters[idxG];
    new_cluster.merge(clusters[(int)idxH]);
    
    clusters[n+i] = new_cluster;
    clusters.erase(idxG);
    clusters.erase(idxH);
}

xt::dump_npy("Z_tan_minimax.npy", Z);